# Loading and Cleaning

**Load Data:**

> Input the pseudomized file name in the " "
e.g. "PseudonymizedData_xxxxxx.xlsx"

In [ ]:
INPUT_FILE = " "

**Clean Data:**

In [ ]:
# @title  {"display-mode":"form"}
# @title
import pandas as pd
import numpy as np
from pathlib import Path

OUTPUT_FILE = "PseudonymizedData_cleaned.xlsx"

# -----------------------------------------------------------
# 2️⃣ Load dataset (auto-detect Excel or CSV)
# -----------------------------------------------------------
input_path = Path(INPUT_FILE)
if not input_path.exists():
    raise FileNotFoundError(f"❌ Cannot find file: {INPUT_FILE}")

print(f"📥 Loading data from: {INPUT_FILE}")
if input_path.suffix.lower() in [".xlsx", ".xls"]:
    df = pd.read_excel(input_path)
elif input_path.suffix.lower() == ".csv":
    df = pd.read_csv(input_path)
else:
    raise ValueError("❌ Unsupported file format. Please use .xlsx, .xls, or .csv")

print(f"✅ Loaded {len(df):,} rows and {len(df.columns)} columns.")

# -----------------------------------------------------------
# 3️⃣ Standardize Column Names
# -----------------------------------------------------------
df.columns = (
    df.columns.str.strip()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

# ✅ Rename registered-date column for consistency
if "Registered_Date" in df.columns:
    df.rename(columns={"Registered_Date": "registered_date"}, inplace=True)
elif "RegisteredDate" in df.columns:
    df.rename(columns={"RegisteredDate": "registered_date"}, inplace=True)
else:
    possible_dates = [c for c in df.columns if "date" in c.lower()]
    if possible_dates:
        print("🔍 Possible date columns found:", possible_dates)
    else:
        raise KeyError("❌ Could not find a date column in the dataset.")

# -----------------------------------------------------------
# 4️⃣ Remove Unnecessary Duplicate Columns
# -----------------------------------------------------------
# Drop alternative repeating hospital code column (if exists)
for col in ["office_hosp_code_1", "office_hosp_code.1"]:
    df = df.drop(columns=[col], errors="ignore")

# -----------------------------------------------------------
# 5️⃣ Normalize Text Fields
# -----------------------------------------------------------
text_cols = df.select_dtypes(include="object").columns
for col in text_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.replace("`", "", regex=False)
        .replace(["None", "none", "NaN", "nan", "N/A", "NA", "n/a", ""], np.nan)
    )

# -----------------------------------------------------------
# 6️⃣ Clean Dates (d/m/yyyy h:mm)
# -----------------------------------------------------------
def clean_date(val):
    """Convert multiple date formats safely with dayfirst=True."""
    if pd.isna(val):
        return np.nan
    try:
        # Expected format dd/mm/yyyy h:mm or d/m/yyyy h:mm
        return pd.to_datetime(val, errors="coerce", dayfirst=True)
    except Exception:
        return np.nan

if "registered_date" in df.columns:
    df["registered_date"] = df["registered_date"].apply(clean_date)
else:
    raise KeyError("❌ 'registered_date' column missing after rename.")

# -----------------------------------------------------------
# 7️⃣ Standardize Common Text Fields
# -----------------------------------------------------------
for col in ["specimen", "site", "specialty", "location"]:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.title()
            .str.replace(r"\s+", " ", regex=True)
            .replace(["Nan", "None"], np.nan)
        )

if "enum_description" in df.columns:
    df["enum_description"] = (
        df["enum_description"]
        .astype(str)
        .str.replace(r"\s+", " ", regex=True)
        .str.replace("No growth after 5 days` incubation", "No growth after 5 days", regex=False)
        .str.replace("No growth after 14 days` incubation", "No growth after 14 days", regex=False)
        .str.replace("`", "", regex=False)
        .str.strip()
        .replace(["Nan", "None"], np.nan)
    )

# -----------------------------------------------------------
# 8️⃣ Remove Duplicate Rows
# -----------------------------------------------------------
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"🧹 Removed {before - after:,} duplicate rows.")

# -----------------------------------------------------------
# 9️⃣ Sort Chronologically by Date (optional)
# -----------------------------------------------------------
if "registered_date" in df.columns:
    df = df.sort_values(by="registered_date")

# -----------------------------------------------------------
# 1️⃣1️⃣ Save Cleaned Data
# -----------------------------------------------------------
df.to_excel(OUTPUT_FILE, index=False)
print(f"\n✅ Cleaning complete!")
print(f"📊 Records retained: {len(df):,}")
print(f"📁 Cleaned dataset saved as: {OUTPUT_FILE}")

# Excel Summary Workbook

In [ ]:
# @title
!pip install xlsxwriter
import pandas as pd
import numpy as np

INPUT_FILE = "PseudonymizedData_cleaned.xlsx"
OUTPUT_FILE = "PseudonymizedData_summary_visualized.xlsx"

df = pd.read_excel(INPUT_FILE)
df.columns = [c.lower() for c in df.columns]

# --------------------------------------------------
# 2️⃣ Data Cleaning
# --------------------------------------------------
if "registered_date" in df.columns:
    df["registered_date"] = pd.to_datetime(df["registered_date"], errors="coerce")

# Safe text replacement (avoids “complex complex”)
if "enum_description" in df.columns:
    df["enum_description"] = df["enum_description"].replace(
        r"(?i)\bKlebsiella\s+pneumoniae\b(?!\s*complex)",
        "Klebsiella pneumoniae complex",
        regex=True,
    )

# --------------------------------------------------
# 3️⃣ Identify Positives & Negatives
# --------------------------------------------------
positive_mask = df["enum_description"].notna() & ~df["enum_description"].str.contains(
    "no growth", case=False, na=False
)
negative_mask = df["enum_description"].str.contains("no growth", case=False, na=False)

# --------------------------------------------------
# 4️⃣ Summary Metrics
# --------------------------------------------------
n_total = len(df)
n_requests = df["request_no"].nunique()
n_positive = df[positive_mask]["request_no"].nunique()
n_negative = df[negative_mask]["request_no"].nunique()
positivity_rate = (
    round(100 * n_positive / (n_positive + n_negative), 2)
    if (n_positive + n_negative) > 0 else np.nan
)

summary_df = pd.DataFrame({
    "Metric": [
        "Total Records",
        "Unique Blood Culture Requests",
        "Positive Requests",
        "Negative Requests",
        "Positivity Rate (%)",
    ],
    "Value": [n_total, n_requests, n_positive, n_negative, positivity_rate],
})

# --------------------------------------------------
# 5️⃣ Organisms (Top 10 for chart)
# --------------------------------------------------
organism_df = (
    df[positive_mask]["enum_description"]
    .value_counts()
    .rename_axis("Organism")
    .reset_index(name="Count")
)
top10_organisms = organism_df.head(10)

# --------------------------------------------------
# 6️⃣ Location
# --------------------------------------------------
if "location" in df.columns:
    location_df = (
        df[positive_mask]
        .groupby("location")["request_no"]
        .nunique()
        .reset_index(name="Positive_Requests")
        .sort_values("Positive_Requests", ascending=False)
        .head(10)
    )
else:
    location_df = pd.DataFrame(columns=["location", "Positive_Requests"])

# --------------------------------------------------
# 7️⃣ Specialty
# --------------------------------------------------
if "specialty" in df.columns:
    specialty_df = (
        df[positive_mask]
        .groupby("specialty")["request_no"]
        .nunique()
        .reset_index(name="Positive_Requests")
        .sort_values("Positive_Requests", ascending=False)
        .head(10)
    )
else:
    specialty_df = pd.DataFrame(columns=["specialty", "Positive_Requests"])

# --------------------------------------------------
# 8️⃣ Monthly Summary (safe, warning-free)
# --------------------------------------------------
if "registered_date" in df.columns:
    df["registered_date"] = pd.to_datetime(df["registered_date"], errors="coerce", dayfirst=True)
    valid_df = df.dropna(subset=["registered_date"]).copy()

    if not valid_df.empty:
        valid_df["month_year"] = valid_df["registered_date"].dt.to_period("M")

        monthly_summary = (
            valid_df.groupby("month_year")
            .apply(
                lambda g: pd.Series({
                    "Total_Requests": g["request_no"].nunique(),
                    "Positives": g.loc[
                        g["enum_description"].notna()
                        & ~g["enum_description"].str.contains("no growth", case=False, na=False),
                        "request_no"
                    ].nunique(),
                    "Negatives": g.loc[
                        g["enum_description"].str.contains("no growth", case=False, na=False),
                        "request_no"
                    ].nunique(),
                }),
                include_groups=False,   # ✅ prevents deprecation warning
            )
            .reset_index()
            .sort_values("month_year")
        )

        monthly_summary["Positivity_Rate(%)"] = (
            100 * monthly_summary["Positives"]
            / (monthly_summary["Positives"] + monthly_summary["Negatives"])
        ).round(2)
        monthly_summary["month_year"] = monthly_summary["month_year"].astype(str)
    else:
        monthly_summary = pd.DataFrame(columns=[
            "month_year", "Total_Requests", "Positives", "Negatives", "Positivity_Rate(%)"
        ])
else:
    monthly_summary = pd.DataFrame(columns=[
        "month_year", "Total_Requests", "Positives", "Negatives", "Positivity_Rate(%)"
    ])

# --------------------------------------------------
# 9️⃣ Positives by Hospital
# --------------------------------------------------
if "office_hosp_code" in df.columns:
    hospital_df = (
        df[positive_mask]
        .groupby("office_hosp_code")["request_no"]
        .nunique()
        .reset_index(name="Positive_Requests")
        .sort_values("Positive_Requests", ascending=False)
    )
else:
    hospital_df = pd.DataFrame(columns=["office_hosp_code", "Positive_Requests"])

# --------------------------------------------------
# 🧾 Capitalize Column Headers
# --------------------------------------------------
def title_case_columns(df):
    df = df.copy()
    df.columns = [c.replace("_", " ").title() for c in df.columns]
    return df

summary_df = title_case_columns(summary_df)
organism_df = title_case_columns(organism_df)
top10_organisms = title_case_columns(top10_organisms)
location_df = title_case_columns(location_df)
specialty_df = title_case_columns(specialty_df)
monthly_summary = title_case_columns(monthly_summary)
hospital_df = title_case_columns(hospital_df)


# --------------------------------------------------
# 1️⃣0️⃣ Excel Output + Charts
# --------------------------------------------------
with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:
    summary_df.to_excel(writer, index=False, sheet_name="Summary")
    organism_df.to_excel(writer, index=False, sheet_name="Top_Organisms")
    location_df.to_excel(writer, index=False, sheet_name="Positives_by_Location")
    specialty_df.to_excel(writer, index=False, sheet_name="Positives_by_Specialty")
    monthly_summary.to_excel(writer, index=False, sheet_name="Positivity_Trend")
    hospital_df.to_excel(writer, index=False, sheet_name="Positives_by_Hospital")

    wb = writer.book
    color_organism = "#3498db"
    color_location = "#1abc9c"
    color_specialty = "#9b59b6"
    color_positive = "#2ecc71"
    color_negative = "#e74c3c"

    # --------------------------------------------------
    # 🥧 PIE – Positive vs Negative
    # --------------------------------------------------
    ws_sum = writer.sheets["Summary"]
    ws_sum.set_column(0, 1, 35)
    pos_row = summary_df.index[summary_df["Metric"] == "Positive Requests"][0]
    neg_row = summary_df.index[summary_df["Metric"] == "Negative Requests"][0]

    pie_chart = wb.add_chart({"type": "pie"})
    pie_chart.add_series({
        "categories": ["Summary", pos_row + 1, 0, neg_row + 1, 0],
        "values": ["Summary", pos_row + 1, 1, neg_row + 1, 1],
        "data_labels": {"percentage": True},
        "points": [{"fill": {"color": color_positive}},
                   {"fill": {"color": color_negative}}],
    })
    pie_chart.set_title({"name": "Positive vs Negative Blood Cultures"})
    ws_sum.insert_chart("E2", pie_chart)
    ws_sum.write("J2", f"Positivity Rate: {positivity_rate}%")

    # --------------------------------------------------
    # 📊 TOP 10 ORGANISMS
    # --------------------------------------------------
    ws_org = writer.sheets["Top_Organisms"]
    ws_org.set_column(0, 1, 45)
    org_chart = wb.add_chart({"type": "column"})
    org_chart.add_series({
        "categories": ["Top_Organisms", 1, 0, len(top10_organisms), 0],
        "values": ["Top_Organisms", 1, 1, len(top10_organisms), 1],
        "data_labels": {"value": True},
        "fill": {"color": color_organism},
    })
    org_chart.set_legend({"none": True})
    org_chart.set_title({"name": "Top 10 Organisms"})
    ws_org.insert_chart("D2", org_chart)

    # --------------------------------------------------
    # 🏥 LOCATION BAR
    # --------------------------------------------------
    ws_loc = writer.sheets["Positives_by_Location"]
    ws_loc.set_column(0, 1, 25)
    loc_chart = wb.add_chart({"type": "bar"})
    loc_chart.add_series({
        "categories": ["Positives_by_Location", 1, 0, len(location_df), 0],
        "values": ["Positives_by_Location", 1, 1, len(location_df), 1],
        "data_labels": {"value": True},
        "fill": {"color": color_location},
    })
    loc_chart.set_legend({"none": True})
    loc_chart.set_title({"name": "Positive Results by Location"})
    ws_loc.insert_chart("D2", loc_chart)

    # 🩺 SPECIALTY BAR
    # --------------------------------------------------
    ws_spec = writer.sheets["Positives_by_Specialty"]
    ws_spec.set_column(0, 1, 25)
    spec_chart = wb.add_chart({"type": "bar"})
    spec_chart.add_series({
        "categories": ["Positives_by_Specialty", 1, 0, len(specialty_df), 0],
        "values": ["Positives_by_Specialty", 1, 1, len(specialty_df), 1],
        "data_labels": {"value": True},
        "fill": {"color": color_specialty},
    })
    spec_chart.set_legend({"none": True})
    spec_chart.set_title({"name": "Positive Results by Specialty"})
    ws_spec.insert_chart("D2", spec_chart)

    # 🏥 HOSPITAL PIE
    # --------------------------------------------------
    ws_hosp = writer.sheets["Positives_by_Hospital"]
    ws_hosp.set_column(0, 1, 25)
    hosp_chart = wb.add_chart({"type": "pie"})
    hosp_chart.add_series({
        "categories": ["Positives_by_Hospital", 1, 0, len(hospital_df), 0],
        "values": ["Positives_by_Hospital", 1, 1, len(hospital_df), 1],
        "data_labels": {"percentage": True},
    })
    hosp_chart.set_title({"name": "Positive Results by Hospital"})
    ws_hosp.insert_chart("D2", hosp_chart)


    # --------------------------------------------------
    # 📈 MONTHLY TREND
    # --------------------------------------------------
    ws_trend = writer.sheets["Positivity_Trend"]
    ws_trend.set_column(0, len(monthly_summary.columns), 20)

    if not monthly_summary.empty:
        month_col = next(
            (c for c in monthly_summary.columns
             if "month" in c.lower() and "year" in c.lower()), None
        )
        formatted_labels = pd.to_datetime(
            monthly_summary[month_col], format="%Y-%m", errors="coerce"
        ).dt.strftime("%b %Y")

        for i, label in enumerate(formatted_labels, start=1):
            ws_trend.write(i, 0, label)

        bar_chart = wb.add_chart({"type": "column"})
        bar_chart.add_series({
            "name": "Total Requests",
            "categories": ["Positivity_Trend", 1, 0, len(monthly_summary), 0],
            "values": ["Positivity_Trend", 1, 1, len(monthly_summary), 1],
            "fill": {"color": "#8AC6D1"},
        })
        bar_chart.set_legend({"none": True})
        bar_chart.set_y_axis({"name": "Total Requests"})

        line_chart = wb.add_chart({"type": "line"})
        line_chart.add_series({
            "name": "Positivity Rate (%)",
            "categories": ["Positivity_Trend", 1, 0, len(monthly_summary), 0],
            "values": ["Positivity_Trend", 1, 4, len(monthly_summary), 4],
            "y2_axis": True,
            "marker": {"type": "circle"},
            "line": {"color": "#F25F5C", "width": 2.5},
            "data_labels": {"value": True},
        })
        line_chart.set_legend({"none": True})
        line_chart.set_y2_axis({"name": "Positivity Rate (%)"})

        bar_chart.combine(line_chart)
        bar_chart.set_title({"name": "Monthly Positivity Rate and Trend"})
        bar_chart.set_x_axis({"name": "Month (Year)", "label_position": "low",
                              "num_font": {"rotation": -45}})

        ws_trend.insert_chart("G2", bar_chart)

print(f"✅ Excel summary workbook created successfully:\n📂 {OUTPUT_FILE}")

# Dashboard

In [ ]:
# @title
variable_name = ""
#!pip install dash plotly pandas
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

EXCEL_FILE = "PseudonymizedData_summary_visualized.xlsx"
sheets = pd.read_excel(EXCEL_FILE, sheet_name=None)

# ------------------------------------------------------------
# Tidy DataFrames
# ------------------------------------------------------------

summary_raw = sheets["Summary"].dropna(how="all", axis=1).copy()
summary_df = (
    summary_raw.iloc[:, :2]
    .rename(columns={"Metric": "metric", "Value": "value"})
    .dropna(subset=["metric"])
)
summary_df["value"] = pd.to_numeric(summary_df["value"], errors="coerce").fillna(summary_df["value"])

organisms_df = (
    sheets["Top_Organisms"]
    .rename(columns={"Organism": "organism", "Count": "count"})
    .dropna(subset=["organism"])
)
organisms_df["count"] = organisms_df["count"].astype(int)

location_df = (
    sheets["Positives_by_Location"]
    .rename(columns={"Location": "location", "Positive Requests": "positive_requests"})
    .dropna(subset=["location"])
)
location_df["positive_requests"] = location_df["positive_requests"].astype(int)

specialty_df = (
    sheets["Positives_by_Specialty"]
    .rename(columns={"Specialty": "specialty", "Positive Requests": "positive_requests"})
    .dropna(subset=["specialty"])
)
specialty_df["positive_requests"] = specialty_df["positive_requests"].astype(int)

hospital_df = (
    sheets["Positives_by_Hospital"]
    .rename(columns={"Office Hosp Code": "office_hosp_code", "Positive Requests": "positive_requests"})
    .dropna(subset=["office_hosp_code"])
)
hospital_df["positive_requests"] = hospital_df["positive_requests"].astype(int)


trend_raw = sheets["Positivity_Trend"].copy()
trend_raw.columns = [c.strip().replace("_", " ").title() for c in trend_raw.columns]
trend_df = trend_raw.rename(
    columns={
        "Month Year": "month_year",
        "Total Requests": "total_requests",
        "Positives": "positives",
        "Negatives": "negatives",
        "Positivity Rate(%)": "positivity_rate",
    }
)
trend_df["month"] = pd.to_datetime(trend_df["month_year"], format="%b %Y", errors="coerce")
trend_df = trend_df.dropna(subset=["month"]).sort_values("month")
for col in ["total_requests", "positives", "negatives", "positivity_rate"]:
    trend_df[col] = pd.to_numeric(trend_df[col], errors="coerce")
# derive numeric year for filter
trend_df["year"] = trend_df["month"].dt.year

print(f"✅ Loaded data from {EXCEL_FILE}")

# ------------------------------------------------------------
# Create figures
# ------------------------------------------------------------

# 🥧 Positive vs Negative
pos_value = summary_df.loc[summary_df["metric"] == "Positive Requests", "value"].iloc[0]
neg_value = summary_df.loc[summary_df["metric"] == "Negative Requests", "value"].iloc[0]
pie_fig = go.Figure(data=[go.Pie(labels=["Positive", "Negative"], values=[pos_value, neg_value], hole=.3)])
pie_fig.update_traces(textinfo="percent+label", marker=dict(colors=["#2ecc71", "#e74c3c"]))
pie_fig.update_layout(title_text="Positive vs Negative Blood Cultures")


# 🧫 Top Organisms
org_fig = px.bar(
    organisms_df.head(10),
    x="organism",
    y="count",
    title="Top 10 Organisms",
    color_discrete_sequence=["#3498db"],
)
org_fig.update_layout(xaxis_title="Organism", yaxis_title="Count", xaxis_tickangle=-45)

# 🏥 Positives by Location
loc_fig = px.bar(
    location_df,
    x="location",
    y="positive_requests",
    title="Positive Results by Location",
    color_discrete_sequence=["#1abc9c"],
)
loc_fig.update_layout(xaxis_title="Location", yaxis_title="Positive Requests", xaxis_tickangle=-45)

# 🩺 Positives by Specialty
spec_fig = px.bar(
    specialty_df,
    x="specialty",
    y="positive_requests",
    title="Positive Results by Specialty",
    color_discrete_sequence=["#9b59b6"],
)
spec_fig.update_layout(xaxis_title="Specialty", yaxis_title="Positive Requests", xaxis_tickangle=-45)

# 🏥 Positives by Hospital Pie
hosp_pie_fig = go.Figure(data=[go.Pie(labels=hospital_df["office_hosp_code"], values=hospital_df["positive_requests"], hole=.3)])
hosp_pie_fig.update_traces(textinfo="percent+label")
hosp_pie_fig.update_layout(title_text="Positive Results by Hospital")


# ------------------------------------------------------------
# Helper: build Monthly Trend figure (filtered)
# ------------------------------------------------------------
def build_trend_figure(selected_year: int):
    dff = trend_df[trend_df["year"] == selected_year]
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Bar(x=dff["month"], y=dff["total_requests"], name="Total Requests", marker_color="#8AC6D1"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(
            x=dff["month"],
            y=dff["positivity_rate"],
            name="Positivity Rate (%)",
            mode="lines+markers",
            line=dict(color="#F25F5C", width=3),
            marker=dict(size=8),
        ),
        secondary_y=True,
    )

    fig.update_layout(
        title_text=f"Monthly Blood Culture Requests & Positivity Rate ({selected_year})",
        xaxis_title="Month",
        yaxis=dict(title="Total Requests"),
        yaxis2=dict(title="Positivity Rate (%)", overlaying="y", side="right"),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        margin=dict(t=70, b=70, l=60, r=60),
    )
    fig.update_xaxes(tickangle=-45)
    return fig

# ------------------------------------------------------------
# Layout
# ------------------------------------------------------------
app = Dash(__name__)
app.title = "Blood Culture Dashboard"

app.layout = html.Div(
    style={"fontFamily": "Arial, sans-serif", "margin": "20px"},
    children=[
        html.H1("🩸 Blood Culture Summary Dashboard",
                style={"textAlign": "center", "color": "#2c3e50"}),

        # Arrange charts in subplots
        html.Div(
            [
                html.Div([dcc.Graph(figure=pie_fig)], style={"flex": "1", "padding": "10px"}),
                html.Div([dcc.Graph(figure=org_fig)], style={"flex": "2", "padding": "10px"}),
            ],
            style={"display": "flex", "flexWrap": "wrap"},
        ),

        html.Div(
            [
                html.Div([dcc.Graph(figure=loc_fig)], style={"flex": "1", "padding": "10px"}),
                html.Div([dcc.Graph(figure=spec_fig)], style={"flex": "1", "padding": "10px"}),
                 html.Div([dcc.Graph(figure=hosp_pie_fig)], style={"flex": "1", "padding": "10px"}),
            ],
            style={"display": "flex", "flexWrap": "wrap"},
        ),


        # --- Year Filter + Trend
        html.Div(
            [
                html.Div(
                    [
                        html.Label("Filter by Year:", style={"fontWeight": "bold"}),
                        dcc.Dropdown(
                            id="year-dropdown",
                            options=[
                                {"label": str(year), "value": year}
                                for year in sorted(trend_df["year"].unique())
                            ],
                            value=sorted(trend_df["year"].unique())[-1],  # default: latest year
                            clearable=False,
                            style={"width": "200px"},
                        ),
                    ],
                    style={"padding": "10px"},
                ),
                html.Div([dcc.Graph(id="trend-graph")]),
            ],
            style={"padding": "10px"},
        ),

        html.Footer(
            f"✅ Data source: {EXCEL_FILE}",
            style={"textAlign": "center", "color": "gray", "marginTop": "25px"},
        ),
    ],
)

# ------------------------------------------------------------
# Callback: update chart based on dropdown
# ------------------------------------------------------------
@app.callback(Output("trend-graph", "figure"),
              Input("year-dropdown", "value"))
def update_trend(selected_year):
    return build_trend_figure(selected_year)

# ------------------------------------------------------------
# Run app
# ------------------------------------------------------------
if __name__ == "__main__":
    app.run(debug=True, port=8051)